In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from pathlib import Path
from time import time, sleep

Task 1
---------
Scrap the USD To EGP Exchange rate from this website
https://www.exchangerates.org.uk/Dollars-to-Egyptian-Pounds-currency-conversion-page.html
and then use it to make a software that takes amount of USD Dollars from the user and calculate how much will it cost in EGP.

In [19]:
def get_amount_in_egp():
    try:
        url='https://www.exchangerates.org.uk/Dollars-to-Egyptian-Pounds-currency-conversion-page.html'
        soup=BeautifulSoup(requests.get(url).text,'html.parser')
        one_usd_amount_in_egp = soup.find(id='shd2b;').get_text()
        return float(one_usd_amount_in_egp)
    except Exception as e:
        return e

try:
    while True:
        usd_amount = float(input('Please enter amount in USD.'))
        one_usd_amount_in_egp = get_amount_in_egp()
        print(str(usd_amount)+" USD is "+str(round(one_usd_amount_in_egp*usd_amount,2))+" EGP")
        option=int(input("Press 1 to continue other to skip."))
        if option != 1:
            break
except Exception as e:
    print(e)

Please enter amount in USD.1
1.0 USD is 18.36 EGP
Press 1 to continue other to skip.1
Please enter amount in USD.15
15.0 USD is 275.35 EGP
Press 1 to continue other to skip.1
Please enter amount in USD.20
20.0 USD is 367.14 EGP
Press 1 to continue other to skip.0


==============================================================================================================

Task 2
---------
Scrap the weather temperature, humidity, visibility, air pressure, wind speed, datetime data from this website
https://eg.freemeteo.com/weather/cairo/current-weather/location/?gid=360630&language=english&country=egypt
for 10 times one read in a minute (so basicly the process takes 10 minutes)
and put them into a CSV & Excel file

In [17]:
def get_weather_data(url):   
    data_lst=[]
    for i in range(0, 10):
        response=requests.get(url) 
        date_time=response.headers['Date'].replace('GMT','') #dataLayer gtag("js", new Date()) 
        soup=BeautifulSoup(requests.get(url).text,'html.parser')
        temp=soup.find('div',attrs={'class':'temp'}).get_text()
        wind=soup.find('div',attrs={'class':'wind-popinfo'}).get_text()
        wind_speed=soup.find('span', attrs={'class':'white'}).get_text().split(':')[1].strip()
        stats=soup.find('div',attrs={'class':'stats'}).contents[0].split('|')
        data=[date_time ,temp, wind, wind_speed]
        for i in stats:
            data.append(i.split(':')[1].strip())
        data_lst.append(data)
        print('Collecting Data ...')
        sleep(60)
    return data_lst


def add_weather_data():
    try:
        url="https://eg.freemeteo.com/weather/cairo/current-weather/location/?gid=360630&language=english&country=egypt"    
        data_lst = get_weather_data(url)
        header=['Datetime','Temperature', 'Wind', 'Wind Speed', 'Humidity', 'Visibility', 'Air Pressure']    
        is_exist = os.path.exists(str(Path().absolute())+'/weather_data.csv')
        if is_exist:
            weather_df=pd.read_csv('datasets/weather_data.csv')
            new_df=pd.DataFrame([data_lst], columns=header)
            updated_df = weather_df.append(new_df, ignore_index=True)
            updated_df.to_csv('weather_data.csv', encoding='utf-8', index=False)
        else:
            df=pd.DataFrame(data_lst, columns=header)
            df.to_csv('weather_data.csv', encoding='utf-8', index=False)
        return pd.read_csv('datasets/weather_data.csv').head(10)
    except Exception as e:
        return(e)
    
add_weather_data()

,Datetime,Temperature,Wind,Wind Speed,Humidity,Visibility,Air Pressure
0,"Wed, 23 Mar 2022 11:48:49",17°C,240°,31 Km/h,30%,> 10000m,1016.0mb
1,"Wed, 23 Mar 2022 11:49:55",17°C,240°,31 Km/h,30%,> 10000m,1016.0mb
2,"Wed, 23 Mar 2022 11:51:00",17°C,240°,31 Km/h,30%,> 10000m,1016.0mb
3,"Wed, 23 Mar 2022 11:52:02",17°C,240°,31 Km/h,30%,> 10000m,1016.0mb
4,"Wed, 23 Mar 2022 11:53:05",17°C,240°,31 Km/h,30%,> 10000m,1016.0mb
5,"Wed, 23 Mar 2022 11:54:07",17°C,240°,31 Km/h,30%,> 10000m,1016.0mb
6,"Wed, 23 Mar 2022 11:55:09",17°C,240°,31 Km/h,30%,> 10000m,1016.0mb
7,"Wed, 23 Mar 2022 11:56:12",17°C,240°,31 Km/h,30%,> 10000m,1016.0mb
8,"Wed, 23 Mar 2022 11:57:15",17°C,240°,31 Km/h,30%,> 10000m,1016.0mb
9,"Wed, 23 Mar 2022 11:58:18",17°C,240°,31 Km/h,30%,> 10000m,1016.0mb


===============================================================================================================

Task 3
---------
Scrap the books (name, price, rate,category) for each category and put them into a CSV & Excel file
https://books.toscrape.com/

In [85]:
def get_categories(url):
    soup=BeautifulSoup(requests.get(url).text,'html.parser')
    categ_lst=soup.find('ul',attrs={'class':''}).find_all('a')
    categ_data=[]
    for categ in categ_lst:
        categ_data.append([categ.get_text().strip(), categ.get('href')])
    return categ_data

    
def get_books(url, categ, data_lst):
    categ_soup=BeautifulSoup(requests.get(url).text,'html.parser')
    books=categ_soup.find_all('article',attrs={'class':'product_pod'})
    for book in books:
        name=book.find('h3').find('a').get('title')
        price=book.find('p', attrs={'class':'price_color'}).get_text()[1:]
        rating=book.find('p', attrs={'class':'star-rating'}).get('class')[1]
        data_lst.append([name, price, rating, categ])
    if categ_soup.find('li', attrs={'class':'next'}):
        next_link = categ_soup.find('li', attrs={'class':'next'}).find('a').get('href')
        url = str(url+str(categ[1]).replace('index.html', next_link))
        data_lst=get_books(url, categ, data_lst)
    return data_lst


def add_categ_book_data():
    header=['Book Name', 'Price', 'Rating', 'Category Name']
    data_lst=[]
    url='https://books.toscrape.com/'
    try:
        categ_data = get_categories(url)
        for categ in categ_data:
            data_lst = get_books(str(url+categ[1]), categ[0], data_lst)
        df = pd.DataFrame(data_lst, columns=header) 
        df.to_csv('books_data.csv', encoding='utf-8', index=False)
        return df.head(10)
    except Exception as e:
        return(e)
    
add_categ_book_data()

,Book Name,Price,Rating,Category Name
0,It's Only the Himalayas,£45.17,Two,Travel
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,£49.43,Four,Travel
2,See America: A Celebration of Our National Par...,£48.87,Three,Travel
3,Vagabonding: An Uncommon Guide to the Art of L...,£36.94,Two,Travel
4,Under the Tuscan Sun,£37.33,Three,Travel
5,A Summer In Europe,£44.34,Two,Travel
6,The Great Railway Bazaar,£30.54,One,Travel
7,A Year in Provence (Provence #1),£56.88,Four,Travel
8,The Road to Little Dribbling: Adventures of an...,£23.21,One,Travel
9,Neither Here nor There: Travels in Europe,£38.95,Three,Travel


In [112]:
df=pd.read_csv('datasets/books_data.csv')
print(len(df))
df.sample(10)

517


,Book Name,Price,Rating,Category Name
400,Psycho: Sanitarium (Psycho #1.5),£36.97,Five,Horror
292,Island of Dragons (Unwanteds #7),£29.65,One,Fantasy
359,Wall and Piece,£44.18,Four,Art
361,Art and Fear: Observations on the Perils (and ...,£48.63,Four,Art
490,If I Gave You God's Phone Number....: Searchin...,£20.91,One,Spirituality
244,Arena,£21.36,Four,Science Fiction
146,Finders Keepers (Bill Hodges Trilogy #2),£53.53,Five,Fiction
372,Civilization and Its Discontents,£59.95,Two,Psychology
15,The Murder of Roger Ackroyd (Hercule Poirot #4),£44.10,Four,Mystery
218,The Coming Woman: A Novel Based on the Life of...,£17.93,Three,Default
